In [34]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import centroid
import os
import shutil
from shapely import MultiPolygon, Polygon, wkt, intersection
import matplotlib.pyplot as plt
from geopy import distance
import pandas_geojson as pdg
import shapely


In [35]:
def get_raw_precinct_gpd(filename='../project_data/chc/ChicagoPrecincts2012_2022.csv'):
    precs = pd.read_csv(filename)
    precs['precinct_id'] = precs.PRECINCT*100 + precs.WARD
    precs['the_geom'] = precs['the_geom'].apply(wkt.loads)
    precs = precs.rename(columns={'the_geom':'geometry'})
    precs = precs.drop(['FULL_TEXT', 'SHAPE_AREA', 'SHAPE_LEN'], axis=1)
    precs['centroid'] = centroid(precs.geometry)

    precs_gpd = gpd.GeoDataFrame(precs, crs='epsg:4326')
    
    return precs_gpd.set_index('precinct_id')

def get_raw_simplex_gpd(filename='../project_data/chc/chc_death_simplices_by_death_in_dim_1.npy'):
    
    dsimps = np.load(filename, allow_pickle=True)
    dsimps_dict = dict(zip(['geometry','death_filtration_value','death_filtration_zscore','death_birth_ratio'],dsimps.T))
    dsimps_gpd = gpd.GeoDataFrame(dsimps_dict, crs='epsg:4326')
    dsimps_gpd['dsimp_index']= dsimps_gpd.index.values

    return dsimps_gpd

def get_raw_polls_gpd(filename = '../project_data/chc/Polling_Places_Chicago_2016.csv'):

    polls = pd.read_csv(filename)
    polls_gpd = gpd.GeoDataFrame()
    polls_gpd['precinct_id'] = polls.Precinct.astype(int)
    polls_gpd['polling_name'] = polls['Polling Place Name']
    polls_gpd['polling_address'] = polls['Address Line 1']
    polls_gpd['polling_description'] = polls['Description']
    polls_gpd['polling_accessible'] = 1 * (polls['Accessible'] == 'Y')
    polls_gpd['polling_zip'] = polls['Zip']
    polls_gpd['polling_lat'] = polls['lat']
    polls_gpd['polling_lon'] = polls['lon']



    return polls_gpd.set_index('precinct_id')


In [36]:
def append_intersecting_simplex_data_to_precinct(precs,dsimps_filename = '../project_data/chc/chc_death_simplices_by_death_in_dim_1.npy'):
    
    dsimps = get_raw_simplex_gpd(dsimps_filename)

    n_intersect = []
    mean_dfv = []
    indices = []

    for ii, prec in precs.iterrows():
        dfvs_ii = []
        indices_ii = []
        
        
        for jj, dsimp in dsimps.iterrows():
    
            if not intersection(prec.geometry,dsimp.geometry).area == 0:
                
                indices_ii.append(str(dsimp.dsimp_index))
                dfvs_ii.append(dsimp.death_filtration_value)
                
        if len(dfvs_ii) == 0:
            mean_dfv.append(0)
            indices.append('')
        else:
            mean_dfv.append(np.mean(dfvs_ii))
            indices.append('_'.join(indices_ii))
                            
        n_intersect.append(len(indices_ii))


    precs['dsimp_n_intersect'] = n_intersect
    precs['dsimp_mean_dfv'] = mean_dfv
    precs['dsimp_indices'] = indices
    return precs

def append_polls_to_precinct(precs, polls_filename = '../project_data/chc/Polling_Places_Chicago_2016.csv'):
    
    polls = get_raw_polls_gpd(polls_filename)

    precs = precs.join(polls)

    imputed = []
    precs_with = precs[precs.polling_lat != -999]
    
    counter = 0
    for ii, prec in precs.iterrows():
        if counter % 100 == 0: print(counter)
        counter += 1
        if prec.polling_lat != -999: 
            imputed.append(0)
            continue
        
        min_dist = None
        nearest_id = None
        for jj, poll in precs_with.iterrows():
   
            dist = distance.distance(prec.centroid.__geo_interface__['coordinates'][::-1],(poll.polling_lat,poll.polling_lon)).km
            if min_dist is None or dist < min_dist:
                min_dist = dist
                nearest_id = jj
        imputed.append(1)
        precs.loc[ii, 'polling_address'] = precs_with.loc[nearest_id].polling_address
        precs.loc[ii, 'polling_name'] = precs_with.loc[nearest_id].polling_name
        precs.loc[ii, 'polling_description'] = precs_with.loc[nearest_id].polling_description
        precs.loc[ii, 'polling_accessible'] = precs_with.loc[nearest_id].polling_accessible
        precs.loc[ii, 'polling_zip'] = precs_with.loc[nearest_id].polling_zip
        precs.loc[ii, 'polling_lat'] = precs_with.loc[nearest_id].polling_lat
        precs.loc[ii, 'polling_lon'] = precs_with.loc[nearest_id].polling_lon
        
    precs['polling_imputed'] = imputed




    return precs
    
def add_census_indices(precs,cen):
    census_indices = []
    for ii, prec in precs.iterrows():
    
        total_area = 0
        prec_area = prec.geometry.area
        indices = []
    
        for jj, block in cen.iterrows():
            area_int = intersection(prec.geometry,block.geometry).area
            if area_int == 0: continue
            else:
                indices.append(str(jj))
                total_area += area_int
        if not len(indices) == 0:
            census_indices.append('_'.join(indices))
        else:
            census_indices.append('')
    precs['census_indices'] = census_indices

    return precs
        
def index_str_to_list(s):
    indices_str = s.split('_')
    if indices_str == ['']:
        return []
    indices = [int(index) for index in indices_str]

    return indices    
    
def average_census_blocks(indices, prec, census_gpd, stat_name):
    prec_area = prec.geometry.area
    total_area = 0
    cum_stat = 0
    total_pop = 0

    for ii in indices:
        tract = census_gpd.iloc[ii].fillna(0)
 
        tract_area = tract.geometry.area
        int_area = intersection(prec.geometry,tract.geometry).area
        frac_tract = int_area/tract_area
        intersection_population = tract.total_population * frac_tract
        stat = tract[stat_name]
        if np.isnan(intersection_population):
            intersection_population = 0
            stat = 0
        total_pop += intersection_population
        total_area += int_area
        weighted_stat = tract[stat_name] * intersection_population

        cum_stat += weighted_stat
    if total_pop == 0:
        cum_stat = total_pop = np.nan
        
    return cum_stat / total_pop, total_pop, total_area/prec_area
        

# NOTE: The census data is missing 16 tracts where there is no median income data. 
# Check total_area/prec_area to see where these missing tracts show up 

def add_census_stat(precs, census_gpd, stat_name):
    
    stats = []
    area_fracs = []
    total_populations = []
    for ii, prec in precs.iterrows():
        indices = index_str_to_list(prec.census_indices)
        stat, total_pop, area_frac = average_census_blocks(indices, prec, cen, stat_name)
        
        stats.append(stat)
        area_fracs.append(area_frac)
        total_populations.append(total_pop)

    if not 'total_population_cen' in precs.columns:
        precs['total_population_cen'] = total_populations
        precs['frac_area_cen'] = area_fracs
    precs['{}_cen'.format(stat_name)] = stats 

    
        

In [43]:
cen = gpd.read_file('../project_data/chc/Chicago_geocensus_data.geojson')
# precs = get_raw_precinct_gpd()
# precs = append_intersecting_simplex_data_to_precinct(precs)
# precs = append_polls_to_precinct(precs)
# precs = add_census_indices(precs,cen)
# precs.to_file('../project_data/processed_Chicago_precinct_data.geojson')     
# DON'T NEED TO RUN ABOVE
precincts = gpd.read_file('../project_data/processed_Chicago_precinct_data.geojson')

In [40]:
cen.columns

Index(['TRACTCE10', 'name', 'total_population', 'median_household_income',
       'white_alone', 'black_alone', 'indigenous_alone', 'asian_alone',
       'pacific_islander_alone', 'other_alone', 'multiracial',
       'work_from_home', 'automobile_to_work', 'public_transit_to_work',
       'taxi_to_work', 'motorcycle_to_work', 'bike_to_work', 'walk_to_work',
       'other_to_work', 'median_age', 'below_poverty_line', 'poverty_total',
       'state', 'county', 'total_pop_20_64_for_employment',
       'in_labor_force_20_64', 'not_in_labor_force_20_64', 'unemployed_20_64',
       '<9th', 'no_diploma', 'high_school', 'some_college', 'associates',
       'bachelors', 'graduate_degree', 'geometry'],
      dtype='object')

We now append each census stat (scaled by things like total population) to `precincts`.

In [47]:
# These stats are scaled by total population
for stat in ['white_alone', 'black_alone', 'indigenous_alone', 'asian_alone',
       'pacific_islander_alone', 'other_alone', 'multiracial',
       'work_from_home', 'automobile_to_work', 'public_transit_to_work',
       'taxi_to_work', 'motorcycle_to_work', 'bike_to_work', 'walk_to_work',
       'other_to_work', 'below_poverty_line', 'poverty_total', '<9th', 
       'no_diploma', 'high_school', 'some_college', 'associates',
       'bachelors', 'graduate_degree']:
    cen['{}_percent'.format(stat)] = cen[stat]/cen['total_population']
    add_census_stat(precincts, cen, '{}_percent'.format(stat))

In [48]:
precincts.columns

Index(['precinct_id', 'WARD', 'PRECINCT', 'centroid', 'dsimp_n_intersect',
       'dsimp_mean_dfv', 'dsimp_indices', 'polling_name', 'polling_address',
       'polling_description', 'polling_accessible', 'polling_zip',
       'polling_lat', 'polling_lon', 'polling_imputed', 'census_indices',
       'geometry', 'total_population_cen', 'frac_area_cen',
       'white_alone_percent_cen', 'black_alone_percent_cen',
       'indigenous_alone_percent_cen', 'asian_alone_percent_cen',
       'pacific_islander_alone_percent_cen', 'other_alone_percent_cen',
       'multiracial_percent_cen', 'work_from_home_percent_cen',
       'automobile_to_work_percent_cen', 'public_transit_to_work_percent_cen',
       'taxi_to_work_percent_cen', 'motorcycle_to_work_percent_cen',
       'bike_to_work_percent_cen', 'walk_to_work_percent_cen',
       'other_to_work_percent_cen', 'below_poverty_line_percent_cen',
       'poverty_total_percent_cen', '<9th_percent_cen',
       'no_diploma_percent_cen', 'high_school_p

In [46]:
#precincts.plot(column='median_household_income_cen',legend=True,legend_kwds={"label": "Median Household Income (Census averaged)"})
